In [1]:
import pandas as pd
import geopandas as gpd
import psycopg2
import os
import random
from source.db_query_split import *

# os.chdir(r'D:\Sociometrik\poc_data_extraction')
os.getcwd()

loc = 'loc.locality, '
day_prt = 'ft.day_part, '
out_loc = r'output_csv'

C:\Users\chemi\Desktop\enviroments\myml\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


## Change your City Name for extract all data also set Start Date + End Date for footfall data

In [2]:
town = "'Faridabad'"  # Use "'Faridabad', 'Gurugram'" for multiple towns
strt_dat = "'2023-06-09'" # Start Date Using for Footfall
end_dat = "'2023-06-10'" # End Date Using for Footfall

### EXTRACT All Data, PoiCount and CS DATA AT TOWN LEVEL

In [3]:
all_dt_town = pd.read_sql(query_splt[1].format('', town), engine)
all_dt_town.to_csv(os.path.join(out_loc,'Alldata_Town_Level.csv'), index=False)
poi_cnt_town = pd.read_sql(query_splt[2].format('', town), engine)
poi_cnt_town.to_csv(os.path.join(out_loc,'POICount_Town_Level.csv'), index=False)
cs_dt_town = pd.read_sql(query_splt[0].format('', town), engine)
cs_dt_town.to_csv(os.path.join(out_loc,'CS_Data_Town_Level.csv'), index=False)

In [4]:
brnd_catg_poi_town = pd.read_sql(query_splt[20].format(town), engine)
brnd_catg_poi_town.to_csv(os.path.join(out_loc,'Brand_POI_CNT_Catgory_Town_Level.csv'), index=False)

### EXTRACT All Data, PoiCount and CS DATA AT LOCALITY LEVEL

In [5]:
all_dt_locality = pd.read_sql(query_splt[1].format(loc, town), engine)
all_dt_locality.to_csv(os.path.join(out_loc,'Alldata_Locality_Level.csv'), index=False)
poi_cnt_locality = pd.read_sql(query_splt[2].format(loc, town), engine)
poi_cnt_locality.to_csv(os.path.join(out_loc,'POICount_Locality_Level.csv'), index=False)
cs_dt_locality = pd.read_sql(query_splt[0].format(loc, town), engine)
cs_dt_locality.to_csv(os.path.join(out_loc,'CS_Data_Locality_Level.csv'), index=False)

In [6]:
brnd_catg_poi_loc = pd.read_sql(query_splt[21].format(town), engine)
brnd_catg_poi_loc.to_csv(os.path.join(out_loc,'Brand_POI_CNT_Catgory_Locality_Level.csv'), index=False)

### EXTRACT FOOTFALL FROM HEX9 TO HEX8 FOR SELECTED TOWN

In [7]:
#--Aggregate from HEX09 to HEX08 by day part and date wise from start date to end date
footfall_hex08_day = pd.read_sql(query_splt[3].format(strt_dat, end_dat, town, day_prt), engine)
footfall_hex08_day.to_csv(os.path.join(out_loc,'FOOTFALL_HEX08_daypart_startDate_endDat.csv'), index=False)

#--Aggregate from HEX09 to HEX08 by date wise from start date to end date
footfall_hex08_dt = pd.read_sql(query_splt[3].format(strt_dat, end_dat, town, ''), engine)
footfall_hex08_dt.to_csv(os.path.join(out_loc,'FOOTFALL_HEX08_startDate_endDat.csv'), index=False)

#--Aggregate from HEX09 to HEX08 all available data
footfall_hex08 = pd.read_sql(query_splt[4].format(town), engine)
footfall_hex08.to_csv(os.path.join(out_loc,'FOOTFALL_HEX08.csv'), index=False)

### EXTRACT FOOTFALL FROM HEX9 TO LOCALITY FOR SELECTED TOWN

In [8]:
#--Aggregate from HEX09 to Locality by day part and date wise from start date to end date
footfall_loc_day = pd.read_sql(query_splt[5].format(strt_dat, end_dat, town, day_prt), engine)
footfall_loc_day.to_csv(os.path.join(out_loc,'FOOTFALL_LOC_daypart_startDate_endDat.csv'), index=False)

#--Aggregate from HEX09 to Locality by date wise from start date to end date
footfall_loc_dt = pd.read_sql(query_splt[5].format(strt_dat, end_dat, town, ''), engine)
footfall_loc_dt.to_csv(os.path.join(out_loc,'FOOTFALL_LOC_startDate_endDat.csv'), index=False)

#--Aggregate from HEX09 to Locality all available data
footfall_loc = pd.read_sql(query_splt[6].format(town), engine)
footfall_loc.to_csv(os.path.join(out_loc,'FOOTFALL_LOC.csv'), index=False)

# -----------------   PART II -------------------

### MAKE 50 METER BUFFER FROM POLYGON AND EXTRACT POI COUNT BY BRAND AND NON BRAND

In [ ]:
# # USE THIS CELL FOR OUTSIDE SHAPEFILE
# # Insert Shapefile into database to make 50 meter buffer and get poi.
# # Make sure your shapefile have columns "property_cd, name, t_name, town_cd, state"
# gdf = gpd.read_file(r'C:\GIS\top104grwthcentre5km_buff.shp')
# #Makesure your table name is not available in DB otherwise it will give you duplicate table name error
# gdf.to_postgis(con = engine_past, name=table_nm, schema='public', if_exists='fail', index=False)

In [9]:
table_nm = 'property'   # Declare Table Name
column_id = 'property_cd, '  # Shapefile table unique id column
column_nm = 'name, '  # shapefile table polygon names column

In [10]:
nmb = random.randint(0,100)
tmp_tbl = str(nmb)+table_nm
conn = psycopg2.connect(pgconnection)
cur = conn.cursor()
cur.execute(query_splt[7].format(table_nm, table_nm, column_id, column_nm))
conn.commit()
buff_brnd_poi_cnt = pd.read_sql(query_splt[8].format(table_nm, column_id, column_nm), engine)
buff_brnd_poi_cnt.to_csv(os.path.join(out_loc,'50Meter_buff_poicnt_brand.csv'), index=False)
cur.execute(query_splt[9].format(table_nm))
conn.commit()
conn.close()

# -----------------   PART III  -------------------

### EXTRACT DATA FROM LOCATION BY GIVEN LAT LONG USING BUFFER

In [2]:
# lat_long = '28.38636, 77.30458', '28.55811, 77.21271'
# lat_long_nm = 'buff0001', 'buff0002'
lat_long = '12.9351396,77.6135028', '12.9745291,77.6073598', '12.9702497,77.6399514','12.9723897,77.6184568', '13.009024,77.5802256', '12.9711526,77.5982523','12.924587,77.5839204', '12.9915916,77.5709741', '12.9323822,77.6313456','12.9120861,77.6507785', '13.0111826,77.5551532', '12.9783723,77.6449235','12.9961241,77.6965574', '12.9569907,77.7321532', '12.981637,77.5637189','12.9823639,77.6107864', '12.8915077,77.5980174', '12.9871672,77.7363763','12.9323763,77.6315128', '12.9728441,77.6074441', '12.9700406,77.6100682','12.9884908,77.5940728', '13.0273999,77.5759234', '13.0193534,77.6429104','12.9858319,77.5877023', '12.99103,77.730332', '12.9239636,77.6714274','12.9074857,77.6011452', '13.0944477,77.5949083', '12.9374443,77.6890184','12.928167,77.6917053', '12.9894485,77.7278413', '12.9200632,77.6851533','12.9060197,77.593834', '13.10197,77.5863591', '12.8915995,77.5980958','12.9962757,77.6954255', '13.0833232,77.6434352', '13.0602352,77.5875447','13.0834994,77.6435871', '12.8885385,77.5629327', '12.9367676,77.6888598'
    

lat_long_nm ='buff0001', 'buff0002', 'buff0003', 'buff0004', 'buff0005', 'buff0006','buff0007', 'buff0008', 'buff0009', 'buff0010', 'buff0011', 'buff0012','buff0013', 'buff0014', 'buff0015', 'buff0016', 'buff0017', 'buff0018','buff0019', 'buff0020', 'buff0021', 'buff0022', 'buff0023', 'buff0024','buff0025', 'buff0026', 'buff0027', 'buff0028', 'buff0029', 'buff0030','buff0031', 'buff0032', 'buff0033', 'buff0034', 'buff0035', 'buff0036','buff0037', 'buff0038', 'buff0039', 'buff0040', 'buff0041', 'buff0042'


buffer = 2000  # Buffer in Meters
strt_dat = "'2023-06-09'" # Start Date Using for Footfall
end_dat = "'2023-06-10'" # End Date Using for Footfall

In [3]:
nmb = random.randint(0,1000)
conn = psycopg2.connect(pgconnection)
cur = conn.cursor()
cur.execute(query_splt[10].format(nmb))
conn.commit()
for i in range(len(lat_long)):
    buff_cd = lat_long_nm[i]
    cur.execute(query_splt[11].format(nmb, buff_cd, lat_long[i], str(buffer)+' Mtr'))
    conn.commit()
cur.execute(query_splt[12].format(nmb))
conn.commit()
cur.execute(query_splt[13].format(nmb, buffer))
conn.commit()

In [ ]:
all_dt_buff = pd.read_sql(query_splt[14].format(nmb, ''), engine)
all_dt_buff.to_csv(os.path.join(out_loc,'AllData_Buff.csv'), index=False)

In [ ]:
poi_cnt_buff = pd.read_sql(query_splt[15].format(nmb, ''), engine)
poi_cnt_buff.to_csv(os.path.join(out_loc,'POICount_Buff.csv'), index=False)

In [ ]:
cs_dt_buff = pd.read_sql(query_splt[16].format(nmb, ''), engine)
cs_dt_buff.to_csv(os.path.join(out_loc,'CS_Buff.csv'), index=False)

In [ ]:
ft_dt_buff = pd.read_sql(query_splt[17].format(nmb, '', strt_dat, end_dat), engine)
ft_dt_buff.to_csv(os.path.join(out_loc,'FootFall_Buff.csv'), index=False)

In [4]:
#-------------------EXTRACT All Data, PoiCount and CS DATA AT BUFFER LEVEL ---------------------------------------------
poi_brnd_buff = pd.read_sql(query_splt[18].format(nmb, ''), engine)
poi_brnd_buff.to_csv(os.path.join(out_loc,'Brand_POICnt_Buff.csv'), index=False)

In [12]:
#-------------------EXTRACT All Data, PoiCount and CS DATA AT BUFFER+LOCALITY LEVEL ------------------------------------
all_dt_buff_loc = pd.read_sql(query_splt[14].format(nmb, loc), engine)
all_dt_buff_loc.to_csv(os.path.join(out_loc,'AllData_Buff_locality.csv'), index=False)
poi_cnt_buff_loc = pd.read_sql(query_splt[15].format(nmb, loc), engine)
poi_cnt_buff_loc.to_csv(os.path.join(out_loc,'POICount_Buff_locality.csv'), index=False)
cs_dt_buff_loc = pd.read_sql(query_splt[16].format(nmb, loc), engine)
cs_dt_buff_loc.to_csv(os.path.join(out_loc,'CS_Buff_locality.csv'), index=False)
ft_dt_buff_loc = pd.read_sql(query_splt[17].format(nmb, loc, strt_dat, end_dat), engine)
ft_dt_buff_loc.to_csv(os.path.join(out_loc,'FootFall_Buff_locality.csv'), index=False)
poi_brnd_buff_loc = pd.read_sql(query_splt[18].format(nmb, loc), engine)
poi_brnd_buff_loc.to_csv(os.path.join(out_loc,'Brand_POICnt_Buff_Locality.csv'), index=False)

In [13]:
#------------ REMOVE EXTRA TABLE FROM DB --------------------------
cur.execute(query_splt[19].format(nmb))
conn.commit()
cur.close()